# Time Series — Univariate Forecast (Sunspots)


# Pinned installs for reproducibility in Colab
!pip -q install "pycaret>=3.0.4,<4" "pandas-datareader>=0.10.0" xgboost lightgbm catboost --upgrade

import os, sys, platform, subprocess, pandas as pd, numpy as np
import matplotlib.pyplot as plt

# Show GPU status (Colab: Runtime -> Change runtime type -> GPU)
try:
    import subprocess
    print(subprocess.check_output(["nvidia-smi"]).decode("utf-8"))
except Exception as e:
    print("No NVIDIA GPU detected (this is OK; PyCaret will fall back to CPU).")

print("PyCaret version check:")
import pycaret
import sklearn
print("pycaret", pycaret.__version__, "| sklearn", sklearn.__version__)


In [ ]:

import pandas as pd
import statsmodels.api as sm
sun = sm.datasets.sunspots.load_pandas().data
# Convert to monthly index to show resampling (original is yearly)
sun.index = pd.period_range(start="1700", periods=len(sun), freq="Y").to_timestamp()
series = sun["SUNACTIVITY"].astype(float).rename("sunspots")

from pycaret.time_series import TSForecastingExperiment
exp = TSForecastingExperiment()
exp.setup(data=series, fh=12, fold=3, session_id=42, seasonal_period=11)

best = exp.compare_models()
final = exp.finalize_model(best)
future = exp.predict_model(final)
exp.plot_model(final, plot="forecast")
future.head()
